# PageRank

In [1]:
import numpy as np

## Traditional Method

In [33]:
def create_transition_matrix(vertices_dict, vertices_order, string = True):
    matrix = []
    for v in vertices_order:
        r = []
        for u in vertices_order:
            if v in vertices_dict[u]:
                if string:
                    r.append("1 / " + str(len(vertices_dict[u])))
                else:
                    r.append(1 / len(vertices_dict[u]))
            else:
                if string:
                    r.append("0")
                else:
                    r.append(0)
        matrix.append(r)
    return matrix


def power_iteration(matrix, node_number, threshold):
    matrix = np.array(matrix)
    count = 0
    old_r = np.ones([node_number, 1]) / node_number
    new_r = matrix.dot(old_r)
    while np.sum(np.abs(new_r-old_r)) >= threshold:
        count += 1
        if (np.sum(np.abs(new_r-old_r))) == 0:
            break
        old_r = new_r
        new_r = matrix.dot(old_r)
    return new_r, count


def power_iteration_by_number(matrix, node_number, iteration_number):
    matrix = np.array(matrix)
    print("transition matrix:")
    print(matrix)
    old_r = np.ones([node_number, 1]) / node_number
    print("initial pagerank:")
    print(old_r)
    new_r = matrix.dot(old_r)
    count = 1
    print("pagerank after", count, "iteration (transition matrix 点乘上一个 pagerank vector):")
    print(new_r)
    while count < iteration_number:
        count += 1
        if (np.sum(np.abs(new_r-old_r))) == 0:
            break
        old_r = new_r
        new_r = matrix.dot(old_r)
        print("pagerank after", count, "iteration (transition matrix 点乘上一个 pagerank vector):")
        print(new_r)
    return new_r, count

In [30]:
nodes = dict()
# nodes["A"] = ["A", "B", "C", "D"]
# nodes["B"] = ["D"]
# nodes["C"] = []
# nodes["D"] = ["B", "C"]
nodes["A"] = ["B", "C", "D"]
nodes["B"] = ["A", "D"]
nodes["C"] = ["A"]
nodes["D"] = ["B", "C"]

nodes_order = ["A", "B", "C", "D"]

transition_matrix = create_transition_matrix(nodes, nodes_order, False)
print("transition matrix:")
for row in transition_matrix:
    print(row)

transition matrix:
[0, 0.5, 1.0, 0]
[0.3333333333333333, 0, 0, 0.5]
[0.3333333333333333, 0, 0, 0.5]
[0.3333333333333333, 0.5, 0, 0]


In [31]:
result = power_iteration(transition_matrix, 4, 0)
print(result)

(array([[0.33333333],
       [0.22222222],
       [0.22222222],
       [0.22222222]]), 53)


In [34]:
after_one = power_iteration_by_number(transition_matrix, 4, 1)
print()
print("final result:")
print(after_one)

transition matrix:
[[0.         0.5        1.         0.        ]
 [0.33333333 0.         0.         0.5       ]
 [0.33333333 0.         0.         0.5       ]
 [0.33333333 0.5        0.         0.        ]]
initial pagerank:
[[0.25]
 [0.25]
 [0.25]
 [0.25]]
pagerank after 1 iteration (transition matrix 点乘上一个 pagerank vector):
[[0.375     ]
 [0.20833333]
 [0.20833333]
 [0.20833333]]

final result:
(array([[0.375     ],
       [0.20833333],
       [0.20833333],
       [0.20833333]]), 1)


## Map Reduce Method

In [35]:
def map_part(url_pagerank_outlinks_dict, node_order):
    result_dict = {}
    for node in node_order:
        result_node_dict = {}
        node_dict = url_pagerank_outlinks_dict[node]
        url_pagerank = list(node_dict.keys())[0]
        outlinks = list(node_dict.values())[0]
        for outlink in outlinks:
            result_node_dict[outlink] = url_pagerank[1] / len(outlinks)
        result_node_dict[url_pagerank[0]] = outlinks
        result_dict[node] = result_node_dict
    return result_dict


def produce_map_output(map_result):
    result_dict = {}
    for node_i in map_result:
        for node_j in map_result[node_i]:
            if node_j.upper() in result_dict:
                result_dict[node_j.upper()].append(map_result[node_i][node_j])
            else:
                result_dict[node_j.upper()] = [map_result[node_i][node_j]]
    return result_dict


def reduce_part(map_result, node_order, d, n):
    result_dict = {}
    for node in node_order:
        node_value = map_result[node]
        pagerank = 0
        value = None
        for v in node_value:
            if type(v) != list:
                pagerank += v
            else:
                value = v
        # 注意
        # pagerank = d * pagerank + (1-d) / n  # 课上的公式
        pagerank = (1 - d) * pagerank + d  # exam 的公式
        key = (node, pagerank)
        result_dict[node] = {key: value}
    return result_dict

In [36]:
url_pagerank_outlinks_dict = {
    "A": {("A", 1/4): ["a", "b", "c", "d"]},
    "B": {("B", 1/4): ["d"]},
    "C": {("C", 1/4): []},
    "D": {("D", 1/4): ["b", "c"]},
}
node_order = ["A", "B", "C", "D"]

print("First Map Input:")
for node in node_order:
    print(node, url_pagerank_outlinks_dict[node])
print()

print("First Map Output:")
map_result = map_part(url_pagerank_outlinks_dict, node_order)
for node in node_order:
    print(node, map_result[node])
print()

print("First Reduce Input:")
map_result = produce_map_output(map_result)
for node in node_order:
    print(node, map_result[node])
print()

print("First Reduce Output:")
reduce_result = reduce_part(map_result, node_order, 0, 4)
for node in node_order:
    print(node, reduce_result[node])


First Map Input:
A {('A', 0.25): ['a', 'b', 'c', 'd']}
B {('B', 0.25): ['d']}
C {('C', 0.25): []}
D {('D', 0.25): ['b', 'c']}

First Map Output:
A {'a': 0.0625, 'b': 0.0625, 'c': 0.0625, 'd': 0.0625, 'A': ['a', 'b', 'c', 'd']}
B {'d': 0.25, 'B': ['d']}
C {'C': []}
D {'b': 0.125, 'c': 0.125, 'D': ['b', 'c']}

First Reduce Input:
A [0.0625, ['a', 'b', 'c', 'd']]
B [0.0625, ['d'], 0.125]
C [0.0625, [], 0.125]
D [0.0625, 0.25, ['b', 'c']]

First Reduce Output:
A {('A', 0.0625): ['a', 'b', 'c', 'd']}
B {('B', 0.1875): ['d']}
C {('C', 0.1875): []}
D {('D', 0.3125): ['b', 'c']}
